In [1]:
import pandas as pd
import os
from datetime import date, datetime
import numpy as np

C:\Users\marie\AppData\Local\Temp\ipykernel_39332\1199149209.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
names = ['animal_number', 'date', 'begin_time', 'end_time', 'yield', 'days_in_milk', 'lactation_number', 'conductivity_lf', 
         'conductivity_lr', 'conductivity_rf', 'conductivity_rr', 'mean_flow_lf', 'mean_flow_lr', 'mean_flow_rf', 'mean_flow_rr', 
         'milk_flow_duration', 'occ', 'peak_flow_lr', 'peak_flow_rf', 'peak_flow_rr', 'peak_flow_lf', 'teats_not_found', 'yield_lf', 
         'yield_lr', 'yield_rf', 'yield_rr', 'interval', 'kickoff', 'kickoff_lf', 'kickoff_lr', 'kickoff_milkings', 'kickoff_rf', 
         'kickoff_rr', 'alternative_attach', 'device_name', 'incomplete', 'is_incomplete', 'incomplete_milkings'] 
dtypes = {'animal_number' : str, 'date' : str, 'begin_time' : str, 'end_time' : str, 'yield' : float, 'days_in_milk' : int, 
          'lactation_number' : int, 'conductivity_lf' : float, 'conductivity_lr' : float, 'conductivity_rf' : float, 
          'conductivity_rr' : float, 'mean_flow_lf' : float, 'mean_flow_lr' : float, 'mean_flow_rf' : float, 'mean_flow_rr' : float, 
          'milk_flow_duration' : str, 'occ' : float, 'peak_flow_lr' : float, 'peak_flow_rf' : float, 'peak_flow_rr' : float, 'peak_flow_lf' : float, 
          'teats_not_found' : str, 'yield_lf' : float, 'yield_lr' : float, 'yield_rf' : float, 'yield_rr' : float, 'interval' : str, 'kickoff' : str, 
          'kickoff_lf' : str, 'kickoff_lr' : str, 'kickoff_milkings' : int, 'kickoff_rf' : str, 'kickoff_rr' : str, 'alternative_attach' : str, 
          'device_name' : str, 'incomplete' : str, 'is_incomplete' : str, 'incomplete_milkings' : int} 


In [8]:
dirpath = os.getcwd()
data_path = os.path.join(dirpath, "..", "data")
file_key = "delpro_visit"
# read data
df23 = pd.read_csv(os.path.join(data_path, f"{file_key}_2023.csv"), delimiter = ";", header = 0,  names = names, dtype = dtypes)
df24 = pd.read_csv(os.path.join(data_path, f"{file_key}_2024.csv"), delimiter = ";", header = 0,  names = names, dtype = dtypes)

df23.milk_flow_duration = [int(num.replace(",", "")) for num in df23.milk_flow_duration]
df24.milk_flow_duration = [int(num.replace(",", "")) for num in df24.milk_flow_duration]

# format dates
df23.date = [datetime.strptime(day, "%m/%d/%Y") for day in df23.date]
df24.date = [datetime.strptime(day, "%m/%d/%Y") for day in df24.date]
# find current lactetion at data collection
coll_date =  datetime(2023, 11, 17)
idx = np.where(df23.date == coll_date)[0]
curr_lact = df23.loc[idx, ["animal_number", "lactation_number"]].drop_duplicates().reset_index(drop=True)

# keep only records from current lactation
new_df23 = pd.merge(left=curr_lact, right=df23,how='left', left_on=["animal_number", "lactation_number"], right_on=["animal_number", "lactation_number"])
new_df24 = pd.merge(left=curr_lact, right=df24,how='left', left_on=["animal_number", "lactation_number"], right_on=["animal_number", "lactation_number"])
year_df = pd.concat([new_df23, new_df24]).drop_duplicates().reset_index(drop=True)

# keep only records from current lactation
new_df23 = pd.merge(left=curr_lact, right=df23,how='left', left_on=["animal_number", "lactation_number"], right_on=["animal_number", "lactation_number"])
new_df24 = pd.merge(left=curr_lact, right=df24,how='left', left_on=["animal_number", "lactation_number"], right_on=["animal_number", "lactation_number"])
year_df = pd.concat([new_df23, new_df24]).drop_duplicates().reset_index(drop=True)

year_df.to_csv(os.path.join(data_path,f"{file_key}_lactation.csv"), index = False)


In [37]:
dim_recs  = year_df[["animal_number", "lactation_number", "days_in_milk", "date"]].drop_duplicates()\
.groupby(["animal_number", "lactation_number"])\
.agg(count_recs = ("days_in_milk", "count"), min_date = ("date", "min"), max_date = ("date", "max"), min_dim = ("days_in_milk", "min"), max_dim = ("days_in_milk", "max"))\
.reset_index()

dim_recs.sort_values(by = "animal_number")

,animal_number,lactation_number,count_recs,min_date,max_date,min_dim,max_dim
0,1003,3,115,2023-07-26,2023-11-17,0.0,114.0
1,1023,4,123,2023-07-23,2023-11-17,0.0,117.0
2,1039,3,306,2023-01-19,2023-11-17,0.0,302.0
3,1040,3,218,2023-04-18,2023-11-17,0.0,213.0
4,1057,4,342,2023-10-28,2024-09-23,0.0,331.0
...,...,...,...,...,...,...,...
234,973,3,336,2023-01-08,2023-12-06,0.0,332.0
235,975,4,328,2023-06-23,2024-05-09,0.0,321.0
236,978,3,309,2023-03-29,2024-01-24,0.0,301.0
237,981,4,25,2023-10-24,2023-11-17,0.0,24.0


In [39]:
year_df.iloc[100844]


animal_number          971
lactation_number         4
date                   NaT
begin_time             NaN
end_time               NaN
yield                  NaN
days_in_milk           NaN
conductivity_lf        NaN
conductivity_lr        NaN
conductivity_rf        NaN
conductivity_rr        NaN
mean_flow_lf           NaN
mean_flow_lr           NaN
mean_flow_rf           NaN
mean_flow_rr           NaN
milk_flow_duration     NaN
occ                    NaN
peak_flow_lr           NaN
peak_flow_rf           NaN
peak_flow_rr           NaN
peak_flow_lf           NaN
teats_not_found        NaN
yield_lf               NaN
yield_lr               NaN
yield_rf               NaN
yield_rr               NaN
interval               NaN
kickoff                NaN
kickoff_lf             NaN
kickoff_lr             NaN
kickoff_milkings       NaN
kickoff_rf             NaN
kickoff_rr             NaN
alternative_attach     NaN
device_name            NaN
incomplete             NaN
is_incomplete          NaN
i